In [ ]:
import numpy as np
import tensorflow as tf
import gym

%matplotlib notebook
import matplotlib.pyplot as plt

Set up the environment and get the dimension of the spaces

In [ ]:
env = gym.make('CartPole-v0')
assert len(env.observation_space.shape) == 1
num_in = env.observation_space.shape[0]
num_actions = env.action_space.n

Define the architecture of the network that will be used

In [ ]:
def mlp_fn(x, num_in, num_hidden, num_out):
    '''Creates an MLP with one hidden layer.'''
    # Create first layer.
    with tf.name_scope('layer1'):
        w1 = tf.Variable(tf.truncated_normal([num_in, num_hidden]), name='weight')
        b1 = tf.Variable(tf.zeros([num_hidden]), name='bias')
        h = tf.nn.relu(tf.matmul(x, w1) + b1)
    # Create second layer.
    with tf.name_scope('layer2'):
        w2 = tf.Variable(tf.truncated_normal([num_hidden, num_out]), name='weight')
        b2 = tf.Variable(tf.zeros([num_out]), name='bias')
        y = tf.matmul(h, w2) + b2
    theta = [w1, b1, w2, b2]
    return y, theta

Instantiate the networks that maps inputs to actions and values.

In [ ]:
state_var = tf.placeholder(tf.float32, shape=(None, num_in), name='state')
# Map the input to the action logits.
with tf.name_scope('policy'):
    logits_op, theta_policy = mlp_fn(state_var, num_in, 20, num_actions)
probs_op = tf.nn.softmax(logits_op)
# Map the input to the scalar value.
with tf.name_scope('value'):
    value_op, theta_value = mlp_fn(state_var, num_in, 20, 1)
theta = theta_policy + theta_value

action_var        = tf.placeholder(tf.int32, shape=(None,), name='action')
future_reward_var = tf.placeholder(tf.float32, shape=(None,), name='future_reward')
num_episodes_var  = tf.placeholder(tf.float32, name='num_episodes_var')

The policy gradient is the gradient of the expected total reward $R$ over the distribution of trajectories $\tau = (s_{1}, a_{1}, s_{2}, a_{2}, \dots)$ defined by the environment and the policy
$$
g = \nabla_{\theta} E_{\tau \sim p_{\theta}}[R]
% = \nabla_{\theta} \sum_{\tau} R p_{\theta}(\tau)
% = \sum_{\tau} R p_{\theta}(\tau) \nabla_{\theta} \log p_{\theta}(\tau)
= E_{\tau \sim p_{\theta}} [R \nabla_{\theta} \log p_{\theta}(\tau)]
$$

The Markov assumption
$$
p(\tau) = p(s_{1}, a_{1}, s_{2}, \dots, s_{n+1})
= p(s_{1}) \left[\prod_{t = 1}^{n} \pi_{\theta}(a_{t} \mid s_{t})\right] \left[\prod_{t = 1}^{n} p(s_{t+1} \mid s_{t}, a_{t})\right]
$$
gives an expression in terms of the policy for
$$
\nabla_{\theta} \log p_{\theta}(\tau) = \sum_{t = 1}^{n} \nabla_{\theta} \log \pi_{\theta}(a_{t} \mid s_{t})
$$
and therefore
$$
g = E_{\tau \sim p_{\theta}} \left[\left(\sum_{t = 1}^{n} r_{t}\right) \left(\sum_{t = 1}^{n} \nabla_{\theta} \log \pi_{\theta}(a_{t} \mid s_{t})\right) \right]
$$
However, since the past rewards are constant with respect to future states and actions, this expression can be modified to depend on only the future rewards
$$
g = E_{\tau \sim p_{\theta}} \left[\sum_{t = 1}^{n} \left(\sum_{k = t}^{n} r_{k}\right) \nabla_{\theta} \log \pi_{\theta}(a_{t} \mid s_{t}) \right]
$$
It is also considered beneficial to subtract a baseline that estimates the value function
$$
g = E_{\tau \sim p_{\theta}} \left[\sum_{t = 1}^{n} \left(\sum_{k = t}^{n} r_{k} - b(s_{t})\right) \nabla_{\theta} \log \pi_{\theta}(a_{t} \mid s_{t}) \right]
$$

Let the action distribution be defined by the softmax over a vector of action scores $f(s; \theta)$
$$
\pi_{\theta}(a \mid s) = \frac{\exp f_{a}(s; \theta)}{\sum_{i} \exp f_{i}(s; \theta)}
$$
then the log likelihood is
$$
-\log \pi_{\theta}(a \mid s) = f_{a}(s; \theta) - \log \sum_{i} \exp f_{i}(s; \theta)
$$
This is the negative of the TensorFlow `sparse_softmax_cross_entropy_with_logits` loss applied to the un-normalized scores.

Finally, we define a loss function whose negative gradient is equal to the policy gradient
$$
L(\theta) = \frac{1}{m} \sum_{i = 1}^{m} \left[\sum_{t = 1}^{n^{(i)}} \left(\sum_{k = t}^{n^{(i)}} r_{t}^{(i)} - b(s_{t}^{(i)})\right) (-1) \log \pi_{\theta}(a_{t} \mid s_{t}) \right]
$$

In [ ]:
def policy_loss_fn(logits, value, action, future_reward, num_eps):
    '''Computes loss function for policy.
    Negative gradient of this function is policy gradient.
    The inputs are per time step, with multiple episodes concatenated.'''
    actions = tf.to_int32(action)
    # Negative log of softmax of logits for sample action.
    neg_log_p = tf.nn.sparse_softmax_cross_entropy_with_logits(logits, action)
    return 1.0/num_eps * tf.reduce_sum(tf.mul(future_reward - value, neg_log_p))

In [ ]:
def run_episode(env, policy, max_time_steps=1000, render=False):
    '''Runs one episode of an OpenAI environment.
    
    policy -- Function that maps observations to action probabilities
        and value function.
    '''
    ep = {
        'state':  [],
        'action': [],
        'reward': [],
        'value':  [],
    }
    s_t = env.reset()
    done = False
    ep['state'].append(s_t)
    for t in xrange(max_time_steps):
        # Evaluate the neural network.
        probs, v_t = policy(s_t)
        ep['value'].append(v_t)
        # Sample the action.
        a_t = np.random.choice(len(probs), p=probs)
        ep['action'].append(a_t)
        s_t, r_t, done, info = env.step(a_t)
        ep['reward'].append(r_t)
        ep['state'].append(s_t)
        if done:
            break
    return ep

In [ ]:
# Define objective.
value_coeff = 1e-2
weight_decay = 1e-3
reward_loss_op = policy_loss_fn(logits_op, value_op, action_var, future_reward_var, num_episodes_var)
value_loss_op = 0.5 * tf.reduce_mean(tf.square(value_op - future_reward_var))
reg_op = sum(tf.nn.l2_loss(x) for x in theta)
loss_op = reward_loss_op + value_coeff*value_loss_op + weight_decay*reg_op

# Define optimizer.
lr = 1e-7
opt = tf.train.MomentumOptimizer(lr, 0.9)
train_op = opt.minimize(loss_op)
grad_op = tf.gradients(loss_op, theta)

In [ ]:
# Gradient descent.

num_iters = 1000
num_episodes = 30

sess = tf.Session()
sess.run(tf.global_variables_initializer())
    
def policy(s):
    p, v = sess.run([probs_op, value_op], feed_dict={
        state_var: np.reshape(s, (1,)+np.shape(s)),
    })
    return p[0], v[0] # Unpack from batch.


def compute_future_reward(r, gamma=1.0):
    '''Computes the cumulative sum from each element to the end.'''
    n = len(r)
    s = 0
    ss = []
    for k in range(n):
        s = r[n-1-k] + gamma*s
        ss.append(s)
    return ss[::-1]

def concat(x):
    return [e for l in x for e in l]

h = {
    'num_episodes': [],
    'reward':       [],
    # Record max-norm of gradients.
    'gradient': [],
    # Record future reward and value estimate.
    'mean_future_reward': [],
    'mean_value':         [],
}

total_num_episodes = 0
for it in xrange(num_iters):
    # Run some episodes and gather data.
    episodes = []
    for i in xrange(num_episodes):
        ep = run_episode(env, policy)
        episodes.append(ep)
        total_num_episodes += 1

    # Compute gradient and take step.
    future_reward = [compute_future_reward(ep['reward']) for ep in episodes]
    _, loss, grads, _ = sess.run([value_op, loss_op, grad_op, train_op], feed_dict={
        state_var:         np.array(concat([ep['state'][:-1] for ep in episodes])),
        action_var:        np.array(concat([ep['action'] for ep in episodes])),
        future_reward_var: np.array(concat(future_reward)),
        num_episodes_var:  float(num_episodes),
    })

    total_reward = [sum(ep['reward']) for ep in episodes]

    h['reward'].append(np.mean(total_reward))
    h['num_episodes'].append(total_num_episodes)
    
    #grad_norms = [np.max(np.abs(g)) for g in grads]
    #h['gradient'].append(grad_norms)
    #h['mean_future_reward'].append(np.mean([np.mean(r) for r in future_reward]))
    #h['mean_value'].append(np.mean([np.mean(ep['value']) for ep in episodes]))
    print '%d  reward:%10.3e loss:%10.3e' % (it, np.mean(total_reward), loss)

fig = plt.figure()
plt.plot(h['reward'])
plt.show()